# Lab 09: Ames Iowa Home Prices

---
author: Your Name Here
date: April 20, 2024
embed-resources: true
---

## Introduction

## Methods

In [265]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from joblib import dump
from sklearn.ensemble import RandomForestRegressor

In [266]:
ames_train = pd.read_csv("https://cs307.org/lab-09/data/ames-train.csv")
ames_test = pd.read_csv("https://cs307.org/lab-09/data/ames-test.csv")

### Data

In [267]:
# create X and y for train dataset
X_train = ames_train.drop("SalePrice", axis=1)
y_train = ames_train["SalePrice"]

# create X and y for test dataset
X_test = ames_test.drop("SalePrice", axis=1)
y_test = ames_test["SalePrice"]

In [268]:
ames_train

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,2832,908188140,160,RM,24.0,2522,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,137500
1,2736,905426150,80,RL,NaN,19690,Pave,NaN,IR1,Lvl,...,738,Gd,GdPrv,NaN,0,8,2006,WD,Alloca,274970
2,2135,907200130,20,RL,97.0,11800,Pave,NaN,IR1,Bnk,...,0,NaN,NaN,NaN,0,8,2007,WD,Family,131000
3,2424,528228415,120,RM,NaN,3072,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,5,2006,WD,Normal,178740
4,1967,535457020,20,RL,80.0,8000,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,11,2007,WD,Normal,156500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,1020,527302070,20,RL,NaN,10825,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,7,2008,WD,Normal,181900
1871,237,905426200,20,RL,65.0,11479,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,144500
1872,1547,910202050,30,RM,40.0,3636,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,1,2008,WD,Normal,55000
1873,1855,533251110,20,RL,80.0,12000,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,3,2007,WD,Normal,255000


In [269]:
# visualizations

### Models

In [270]:
def mape(y_true, y_pred):
    return (abs(y_true - y_pred) / y_true).mean()

In [271]:
X_train.dtypes

Order               int64
PID                 int64
MS SubClass         int64
MS Zoning          object
Lot Frontage      float64
                   ...   
Misc Val            int64
Mo Sold             int64
Yr Sold             int64
Sale Type          object
Sale Condition     object
Length: 81, dtype: object

In [459]:
numeric_features = X_train.select_dtypes(include=["float64", "int64"]).columns
categorical_features = X_train.select_dtypes(include=["object"]).columns
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest, f_regression

numeric_transformer = Pipeline(
    steps=[
        ("KNN Imputer", KNNImputer(n_neighbors=5)),  
        ("Standardization", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("Modal Imputer", SimpleImputer(strategy="most_frequent")),
        ("One-Hot Encoder", OneHotEncoder(handle_unknown='ignore')),  # Set to ignore unknown categories
    ]
)

# Modify the ColumnTransformer to use the new transformers
preprocessor = ColumnTransformer(
    transformers=[
        ("Numeric Transformer", numeric_transformer, numeric_features),
        ("Categorical Transformer", categorical_transformer, categorical_features),
    ],
    remainder="drop",  # Use "passthrough" to keep all other columns not specified in "transformers"
)

# Define the model pipeline with the RandomForestRegressor
model_pipeline = Pipeline(
    steps=[
        ("Preprocessor", preprocessor),
        ("Model", RandomForestRegressor(
            random_state=4, 
            max_depth=19, 
            min_samples_split=2, 
            min_samples_leaf=1, 
            n_estimators=100, 
            bootstrap=True
        )), 
    ]
)


In [460]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('Preprocessor',
                 ColumnTransformer(transformers=[('Numeric Transformer',
                                                  Pipeline(steps=[('KNN '
                                                                   'Imputer',
                                                                   KNNImputer()),
                                                                  ('Standardization',
                                                                   StandardScaler())]),
                                                  Index(['Order', 'PID', 'MS SubClass', 'Lot Frontage', 'Lot Area',
       'Overall Qual', 'Overall Cond', 'Year Built', 'Year Remod/Add',
       'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF',
       'Total Bsmt SF',...
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating', 'Heating QC',
       'Central Air', 'Electrical', 'Kitchen Qual', 'Functional',
       'Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual',
       'Garage Cond', 'Paved Drive', 'Pool QC', 'Fence', 'Misc Feature',
       'Sale Type', 'Sale Condition'],
      dtype='object'))])),
                ('Model', RandomForestRegressor(max_depth=19, random_state=4))])

In [461]:
# predict on test data
y_pred = model_pipeline.predict(X_test)
mape(y_test, y_pred)

0.0895606952966827

In [463]:
dump(model_pipeline, "ames-housing.joblib", compress=9)

['ames-housing.joblib']